In [1]:
# https://qiita.com/SaitoTsutomu/items/f4478dfbc3c1cf6425e3
import numpy as np, pandas as pd


In [2]:
columns = ["Controller", "Promoter", "Supporter", "Analyzer"]
index = ["P", "Q", "R", "S", "T", "U"]
scores = pd.DataFrame([
    [6,0,1,3],
    [2,-1,3,5],
    [2,4,0,0],
    [3,4,5,0],
    [0,2,1,4],
    [2,3,-1,1]
], columns=columns, index=index)

num_teams = 3
num_members = scores.shape[0]
print(f"makes {num_teams} teams with {num_members} members")

makes 3 teams with 6 members


In [3]:
scores

,Controller,Promoter,Supporter,Analyzer
P,6,0,1,3
Q,2,-1,3,5
R,2,4,0,0
S,3,4,5,0
T,0,2,1,4
U,2,3,-1,1


In [9]:
from pulp import LpProblem, lpSum, value, lpDot
from ortoolpy import addvars, addbinvars

In [7]:
m = LpProblem()
x = np.array(addbinvars(num_members, num_teams)) # Allocations
y = np.array(addvars(num_teams, 2)) # Minimum and maximum within a team
z = addvars(2) # Minimum and maximum across all teams

# objective function
# Minimize the sum of:
# - difference between the highest and lowest scores within a team (y)
# - difference between the highest and lowest scores across all teams (z*1.5)
m += lpSum(y[:,1] - y[:,0]) + 1.5 * (z[1] - z[0] )


In [8]:

# Each members needs to be in exactly one team
# vector of team allocation of a member must be 1.
for i in range(num_members):
    m += lpSum(x[i]) == 1

In [10]:
for j in range(num_teams):

    # scores.sum(1) is vector of scores of a member
    # x[:,j] is vector of team allocation of a member
    # x[0] is minimum score across all teams
    # x[1] is maximum score across all teams
    m += lpDot(scores.sum(1), x[:,j]) >= z[0]
    m += lpDot(scores.sum(1), x[:,j]) <= z[1]

    # Iterate over 4 types of all communication skills
    for k in range(scores.shape[1]):
        # scores.iloc[:,k] is vector of scores of a member for a skill
        # x[:,j] is vector of team allocation of a member
        # y[j, 0] is minimum score within a team
        m += lpDot(scores.iloc[:,k], x[:,j]) >= y[j, 0]
        m += lpDot(scores.iloc[:,k], x[:,j]) <= y[j, 1]

In [12]:
m.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/toshiya/Library/Caches/pypoetry/virtualenvs/combinatorial-optimization-practice-O_WzHL3X-py3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_x/8s9ykbp91d39v67y3yj2j_fc0000gn/T/08b1de3f9faa44fb9a26acf0dd786d9c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/_x/8s9ykbp91d39v67y3yj2j_fc0000gn/T/08b1de3f9faa44fb9a26acf0dd786d9c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41 COLUMNS
At line 284 RHS
At line 321 BOUNDS
At line 340 ENDATA
Problem MODEL has 36 rows, 26 columns and 198 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6 - 0.00 seconds
Cgl0004I processed model has 36 rows, 26 columns (18 integer (18 of which binary)) and 198 elements
Cbc0038I Initial state - 13 integers unsatisfied sum - 3.89673
Cbc0038I Pass   1: suminf.

1

In [ ]:
# x is allocation of members to teams
# m.resolve changes the allocations in the x variable
result_x = np.vectorize(value)(x)
teams = ["A", "B", "C"]
print([teams[i] for i in (result_x @ range(num_teams)).astype(int)])

In [14]:
# @ is matrix multiplication
result_x @ range(num_teams)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)